<a href="https://colab.research.google.com/github/bystrowska/idiom-paraphrasing/blob/main/idiom_paraphrasing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install datasets transformers

     |████████████████████████████████| 4.0 MB 9.4 MB/s 
     |████████████████████████████████| 895 kB 48.6 MB/s 
     |████████████████████████████████| 6.6 MB 42.5 MB/s 
     |████████████████████████████████| 596 kB 36.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Load the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files="drive/MyDrive/Colab data/data.csv")
dataset

Using custom data configuration default-31c18aea220c8b74
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-31c18aea220c8b74/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Idiom', 'Sense', 'Idiomatic_Sent', 'Literal_Sent', 'Idiomatic_Label', 'Literal_Label'],
        num_rows: 5170
    })
})

In [14]:
sample = dataset["train"].shuffle(seed=42).select(range(100))
sample[:3]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-31c18aea220c8b74/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-835146899679acbf.arrow


{'Idiom': ['bundle up', 'lovey-dovey', 'under no illusions'],
 'Idiomatic_Label': ['O B O I O O O O O O O O O O O O O O',
  'O O O O O O O O O O O O O O O O O O B O O O O O O O O O O O O O O O O',
  'O O B I I I I O O O O O O O O'],
 'Idiomatic_Sent': ['I bundled Andy up before allowing him to go outside and They are expecting freezing temperatures tonight .',
  "I always used to think that Dan was a very reserved person , but after watching him get lovey-dovey with his girlfriend at the club , I think there 's another side to him .",
  'The mayor was under no illusion about the quality of infrastructure of the city .'],
 'Literal_Label': ['O B O I O O O O O O O O O O O O O O',
  'O O O O O O O O O O O O O O O O O O B I O O O O O O O O O O O O O O O O',
  'O O B O I O O O O O O O'],
 'Literal_Sent': ['I dressed Andy warmly before allowing him to go outside and They are expecting freezing temperatures tonight .',
  "I always used to think that Dan was a very reserved person , but after 

In [15]:
print("number of sentences: " + str(len(dataset["train"])))
print("number of unique idioms: " + str(len(dataset["train"].unique("Idiom"))))

number of sentences: 5170
number of unique idioms: 823


Split the dataset into training and validate sets.

The `train_test_split` method shuffles the data and splits it into `train` and `test`. In order to get `train`, `test` and `validate` I'll first split it into `test` and "the rest" and then split the rest into `train` and `validate`.

The book we used in ML module last semester recommends 50:25:25 split if there's plenty of data and 60:20:20 otherwise. Since I don't have a lot of data I'll go with the second.

1) let's split data into **train** and **test** with the split being 80:20 -> test is 20% of the whole


In [16]:
split_dataset = dataset['train'].train_test_split(test_size=0.2)

In [18]:
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['Idiom', 'Sense', 'Idiomatic_Sent', 'Literal_Sent', 'Idiomatic_Label', 'Literal_Label'],
        num_rows: 4136
    })
    test: Dataset({
        features: ['Idiom', 'Sense', 'Idiomatic_Sent', 'Literal_Sent', 'Idiomatic_Label', 'Literal_Label'],
        num_rows: 1034
    })
})


2) let's split `dataset[train]` into `test` and `validate` -> validate is meant to be 20% of the whole, so 25% of the current train set

In [21]:
idk = split_dataset['train'].train_test_split(test_size=0.25)
split_dataset['train'] = idk['train']
split_dataset['validate'] = idk['test']

In [22]:
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['Idiom', 'Sense', 'Idiomatic_Sent', 'Literal_Sent', 'Idiomatic_Label', 'Literal_Label'],
        num_rows: 3102
    })
    test: Dataset({
        features: ['Idiom', 'Sense', 'Idiomatic_Sent', 'Literal_Sent', 'Idiomatic_Label', 'Literal_Label'],
        num_rows: 1034
    })
    validate: Dataset({
        features: ['Idiom', 'Sense', 'Idiomatic_Sent', 'Literal_Sent', 'Idiomatic_Label', 'Literal_Label'],
        num_rows: 1034
    })
})


In [26]:
split_dataset.save_to_disk("drive/MyDrive/Colab data/clean_dataset") #saves dataset in arrow format

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-31c18aea220c8b74/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-5ee86d3e33f885c6.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-31c18aea220c8b74/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-a98b5f8cb04d583b.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-31c18aea220c8b74/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-06cb6f5eeb52312b.arrow


#Tokenizing

In [24]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small") # using t5-small for testing to hopefully save time


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [28]:
from datasets import load_from_disk

dataset = load_from_disk("drive/MyDrive/Colab data/clean_dataset")
dataset

DatasetDict({
    train: Dataset({
        features: ['Idiom', 'Sense', 'Idiomatic_Sent', 'Literal_Sent', 'Idiomatic_Label', 'Literal_Label'],
        num_rows: 3102
    })
    test: Dataset({
        features: ['Idiom', 'Sense', 'Idiomatic_Sent', 'Literal_Sent', 'Idiomatic_Label', 'Literal_Label'],
        num_rows: 1034
    })
    validate: Dataset({
        features: ['Idiom', 'Sense', 'Idiomatic_Sent', 'Literal_Sent', 'Idiomatic_Label', 'Literal_Label'],
        num_rows: 1034
    })
})

In [32]:
def f_tokenize(example):
  return tokenizer(example["Idiomatic_Sent"])

tokenized_dataset = dataset.map(f_tokenize, batched=True)

Loading cached processed dataset at drive/MyDrive/Colab data/clean_dataset/train/cache-925ca5b98a7a65a1.arrow
Loading cached processed dataset at drive/MyDrive/Colab data/clean_dataset/test/cache-1cbb855f966959ab.arrow
Loading cached processed dataset at drive/MyDrive/Colab data/clean_dataset/validate/cache-09c9cd3c7a4f306c.arrow


{'Idiom': ['drive up the wall', 'carrot and stick', 'in a huff'],
 'Idiomatic_Label': ['O O O O O O O O B O I I I O',
  'O O O B I O I I O O O O O O',
  'O O O B I I O O O O O O O O O'],
 'Idiomatic_Sent': ['The loud music played by my neighbours is driving me up the wall .',
  'Some schools prefer the carrot and stick method when it comes to education .',
  'People who are in a huff about the Met Gala theme are being absurd .'],
 'Literal_Label': ['O O O O O O O O B O O',
  'O O O O B I I I O I I I I O O O O O O',
  'O O O B O O O O O O O O O'],
 'Literal_Sent': ['The loud music played by my neighbours is irritating me .',
  'Some schools prefer to reward students for good and punish them for bad when it comes to education .',
  'People who are upset about the Met Gala theme are being absurd .'],
 'Sense': ['to make someone angry or irritated',
  'mixture of rewards and punishments',
  'to be upset or angry about something'],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [34]:
tokenized_dataset.save_to_disk("drive/MyDrive/Colab data/tokenized_dataset") #saves dataset in arrow format

#TensorFlow stuff
Import tensorflow:

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)


Tensorflow version 2.8.0


In order to speed up training TPU can be used, to do so execute the cell below.

In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.8.0


BaseException: ignored